## import

In [757]:
import pandas as pd
import numpy as np
import json
import snownlp
from snownlp import SnowNLP
from sklearn import preprocessing
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Dropout
from numpy import linalg as LA

## import data

In [13]:
data= pd.read_csv("ted_main.csv")
data.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [282]:
EDA_data=data[['title','description','event','duration','languages','tags','ratings','views']]
EDA_data

,title,description,event,duration,languages,tags,ratings,views
0,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,1164,60,"['children', 'creativity', 'culture', 'dance',...","[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",47227110
1,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,977,43,"['alternative energy', 'cars', 'climate change...","[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",3200520
2,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,1286,26,"['computers', 'entertainment', 'interface desi...","[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",1636292
3,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,1116,35,"['MacArthur grant', 'activism', 'business', 'c...","[{'id': 3, 'name': 'Courageous', 'count': 760}...",1697550
4,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,1190,48,"['Africa', 'Asia', 'Google', 'demo', 'economic...","[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",12005869
5,Why we do what we do,"Tony Robbins discusses the ""invisible forces"" ...",TED2006,1305,36,"['business', 'culture', 'entertainment', 'goal...","[{'id': 7, 'name': 'Funny', 'count': 1102}, {'...",20685401
6,Letting go of God,When two young Mormon missionaries knock on Ju...,TED2006,992,31,"['Christianity', 'God', 'atheism', 'comedy', '...","[{'id': 3, 'name': 'Courageous', 'count': 477}...",3769987
7,Behind the design of Seattle's library,Architect Joshua Prince-Ramus takes the audien...,TED2006,1198,19,"['architecture', 'collaboration', 'culture', '...","[{'id': 9, 'name': 'Ingenious', 'count': 195},...",967741
8,Let's teach religion -- all religion -- in sch...,Philosopher Dan Dennett calls for religion -- ...,TED2006,1485,32,"['God', 'TED Brain Trust', 'atheism', 'brain',...","[{'id': 3, 'name': 'Courageous', 'count': 562}...",2567958
9,A life of purpose,"Pastor Rick Warren, author of ""The Purpose-Dri...",TED2006,1262,31,"['Christianity', 'God', 'culture', 'happiness'...","[{'id': 21, 'name': 'Unconvincing', 'count': 8...",3095993


## get important event

In [670]:
impute_grps = EDA_data.pivot_table(values=["views"], index=["event"], aggfunc=np.mean)
print(impute_grps)
top_ten_event=impute_grps.views.nlargest(10).index
EDA_data.loc[~EDA_data.event.isin(top_ten_event),'event']='others'
EDA_data.event

                                views
event                                
Stanford University      4.658636e+06
TEDxBloomington          9.484260e+06
TEDxCHUV                 4.682458e+06
TEDxCreativeCoast        8.444981e+06
TEDxGoldenGatePark 2012  4.811180e+06
TEDxHouston              1.614025e+07
TEDxLinnaeusUniversity   4.984884e+06
TEDxNorrkoping           6.569493e+06
TEDxPuget Sound          3.430943e+07
TEDxUW                   5.917201e+06
others                   1.655646e+06


c:\program files\python35\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0       others
1       others
2       others
3       others
4       others
5       others
6       others
7       others
8       others
9       others
10      others
11      others
12      others
13      others
14      others
15      others
16      others
17      others
18      others
19      others
20      others
21      others
22      others
23      others
24      others
25      others
26      others
27      others
28      others
29      others
         ...  
2520    others
2521    others
2522    others
2523    others
2524    others
2525    others
2526    others
2527    others
2528    others
2529    others
2530    others
2531    others
2532    others
2533    others
2534    others
2535    others
2536    others
2537    others
2538    others
2539    others
2540    others
2541    others
2542    others
2543    others
2544    others
2545    others
2546    others
2547    others
2548    others
2549    others
Name: event, Length: 2550, dtype: object

In [738]:
one_hot_event=pd.get_dummies(EDA_data.event)
one_hot_event
#pd.concat([EDA_data, one_hot_event], axis=1)

,Stanford University,TEDxBloomington,TEDxCHUV,TEDxCreativeCoast,TEDxGoldenGatePark 2012,TEDxHouston,TEDxLinnaeusUniversity,TEDxNorrkoping,TEDxPuget Sound,TEDxUW,others
0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,1


## get title/describetion sentiment 

In [625]:
def to_sentiment(description):
    s=SnowNLP(description)
    return(s.sentiments)
description_sentiment=[]
nonlinear_description_sentiment=[]
title_sentiment=[]
nonlinear_title_sentiment=[]
for description in EDA_data.description:
    description_sentiment.append(to_sentiment(description))
    nonlinear_description_sentiment.append(1/to_sentiment(description))
for title in EDA_data.title:
    title_sentiment.append(to_sentiment(title))
    nonlinear_title_sentiment.append(1/to_sentiment(title))
description_sentiment=pd.Series(description_sentiment)
nonlinear_description_sentiment=pd.Series(nonlinear_description_sentiment)
title_sentiment=pd.Series(title_sentiment)
nonlinear_title_sentiment=pd.Series(nonlinear_title_sentiment)
print(nonlinear_title_sentiment.max())


148.075405886


array([[ 1.        ,  0.04377422, -0.00559754],
       [ 0.04377422,  1.        , -0.00498338],
       [-0.00559754, -0.00498338,  1.        ]])

## clean up tags

In [760]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def map_new(x,true_tags):
    max_match=0
    max_index = 0
    for i in true_tags:
        if similar(i,x)>max_match:
            max_match = similar(i,x)
            max_index = i
    print(max_index)
    return(max_index)

def get_flat_tags(df):
    tags=[]
    for tag in df.tags:
        old=tag[1:len(df.tags[1])-1]
        nospace=old.replace(" ", "")
        clean=nospace.replace("'", "")
        clean = clean.replace("]","")
        tags.append(clean.split(","))
    df['clean_tags']=tags
    flat_tags = sum(tags,[])
    return(flat_tags)

def get_unique_tags(flat_tags):
    unique_tags = pd.Series(flat_tags).unique()
    unique_tags = sorted(unique_tags)
    return(unique_tags)

def get_true_tags(unique_tags):
    tag_num = len(unique_tags)
    true_tags = []
    for i in range(0,tag_num-1):
        if similar(unique_tags[i],unique_tags[i+1])<0.6:
            true_tags.append(unique_tags[i+1])
    return(true_tags)
            
def append_new_tags(df,true_tags):
    new_tags = []
    for tagings in df.clean_tags:
        new_tag = []
        for tag in tagings:
            new_tag.append(map_new(tag,true_tags))
        new_tags.append(new_tag)
    return(new_tags)

In [306]:
unique_tags = get_unique_tags(EDA_data)

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['',
 '"Alzheimers"',
 '3dprinting',
 'AI',
 'AIDS',
 'Addiction',
 'Africa',
 'Anthropocene',
 'Asia',
 'Autismspectrumdisorder',
 'Bioethics',
 'Blindness',
 'Brand',
 'Brazil',
 'Buddhism',
 'CRISPR',
 'Christianity',
 'CriminalJustice',
 'DNA',
 'Debate',
 'Egypt',
 'Europe',
 'ForeignPolicy',
 'Genderequality',
 'Genderspectrum',
 'God',
 'Google',
 'Guns',
 'HIV',
 'Humanbody',
 'Internet',
 'Iran',
 'Islam',
 'LGBT',
 'MacArthurgrant',
 'Mars',
 'MiddleEast',
 'Moon',
 'NASA',
 'Naturalresources',
 'NewYork',
 'Nobelprize',
 'PTSD',
 'Planets',
 'Senses',
 'Slavery',
 'SouthAmerica',
 'Stringtheory',
 'Surgery',
 'Surveillance',
 'Syria',
 'TED-Ed',
 'TEDBooks',
 'TEDBrainTrust',
 'TEDFellows',
 'TEDMED',
 'TEDNYC',
 'TEDPrize',
 'TEDResidency',
 'TEDYouth',
 'TEDenEspañol',
 'TEDx',
 'Transgender',
 'UnitedStates',
 'Vaccines',
 'activism',
 'adventure',
 'advertising',
 'aging',
 'agriculture',
 'aircraft',
 'algorithm',
 'alternativeenergy',
 'ancientworld',
 'animals',
 'ani

In [315]:
true_tags = get_true_tags(unique_tags)
true_tags
len(true_tags)

394

In [759]:
new_tags = append_new_tags(EDA_data,true_tags)

children
creativity
culture
dance
educat
marketing
hearing
alternativeenergy
cars
climatechange
culture
environm
globalis
rocketscience
disability
geology
computers
entertainment
industrialdesign
medi
music
performanc
simplicity
software
geology
MacArthurgrant
activism
business
cities
environm
green
Genderequality
poli
illusion
Africa
Asia
Google
demo
economi
globalis
globalis
hearthealth
math
statistic
visualiz
business
culture
entertainment
goal-setting
motiv
pote
psychology
Christianity
God
atheism
comedy
culture
humor
performanc
religion
storyte
architecture
collaboration
culture
depression
library
God
TEDBrainTrust
atheism
brain
cognitivescience
consciousness
corruption
philanthrop
religion
Christianity
God
culture
apes
leadership
motiv
philanthrop
religion
activism
architecture
collaboration
culture
depression
disasterrelief
globalis
environm
open-source
p
TEDPrize
art
culture
entertainment
film
globalis
movies
peace
climatechange
storyte
TEDPrize
collaboration
diseas
ebola
globa

KeyboardInterrupt: 

## get important tags

In [776]:
from collections import Counter
flat_tags = sum(new_tags,[])
count_tags=Counter(flat_tags)
Counter_true_tags = count_tags.most_common(50)
ten_most = list(map(lambda x:(x[0]),Counter_true_tags))

def find_popular(x):
    temp = []
    for i in x:
        if i not in ten_most:
            temp.append('others')
        else:
            temp.append(i)
    return temp
#     EDA_data.loc[~x.isin(ten_most),'clean_tags']='others'
#     x = list(x)

main_tags=list(map(find_popular,list(new_tags)))
Counter_true_tags

[('culture', 666),
 ('geology', 623),
 ('globalis', 543),
 ('TEDx', 450),
 ('rocketscience', 445),
 ('depression', 443),
 ('business', 394),
 ('entertainment', 374),
 ('communication', 337),
 ('environm', 277),
 ('hearthealth', 241),
 ('biology', 224),
 ('climatechange', 223),
 ('art', 222),
 ('conservation', 216),
 ('creativity', 203),
 ('collaboration', 173),
 ('economi', 163),
 ('humanit', 162),
 ('biodiversity', 158),
 ('activism', 157),
 ('brain', 157),
 ('societ', 157),
 ('educat', 152),
 ('children', 144),
 ('TEDFellows', 143),
 ('theater', 143),
 ('data', 138),
 ('music', 138),
 ('mentalhealth', 137),
 ('poli', 137),
 ('medici', 135),
 ('performanc', 131),
 ('histo', 129),
 ('cities', 123),
 ('demo', 120),
 ('computers', 111),
 ('Genderequality', 111),
 ('storyte', 110),
 ('animals', 109),
 ('engineering', 108),
 ('psychology', 106),
 ('humor', 105),
 ('Africa', 105),
 (0, 102),
 ('exploration', 99),
 ('typography', 92),
 ('war', 90),
 ('life', 90),
 ('philanthrop', 85)]

## one hot encode tags

In [247]:
def to_onehot(tags):
    sum_tags = sum(tags,[])
    unique_tags = list(set(sum_tags))
    sum_tags
    # 2. LabelEncoder
    le = preprocessing.LabelEncoder()
    
    le.fit(sum_tags)
    print("Label fit all tags")
    sum_tags_2 = le.fit_transform(sum_tags)
    print("all tag Label",sum_tags_2.shape)
    # print(list(le.classes_))
    encode_tags=[]
    i=0
    for x in tags :
        x=(le.transform(x))
        encode_tags.append(x)
        i=i+1
        if i%100==0:
            print("encode num ",i," done")
    # 2. onehot
    enc = preprocessing.OneHotEncoder()
    print("onehot fit all tags")
    enc.fit(sum_tags_2.reshape(17161,1))
    onehotlabels = enc.transform(sum_tags_2.reshape(17161,1)).toarray()
    print("all tag one hot",onehotlabels.shape)
    one_hot_tags=[]
    i=0
    for x in encode_tags:
        onehotlabels = enc.transform(x.reshape(-1,1)).toarray()
        one_hot_tags.append(onehotlabels)
        i=i+1
        if i%100==0:
            print("encode num ",i," done\nresults:",onehotlabels)
    return(one_hot_tags)

In [280]:
def sumtags(one_hot_tags):
    one_hot_sum_tags=[]
    for aTadTalk in one_hot_tags:
        x=0
        for tag in aTadTalk:
            x=x+tag
#         print(x.sum())
        one_hot_sum_tags.append(x)
    return (one_hot_sum_tags)

In [777]:
one_hot_tags=to_onehot(main_tags)
one_hot_sum_tags=sumtags(one_hot_tags)
one_hot_sum_tags=pd.DataFrame(np.array(one_hot_sum_tags))

Label fit all tags
all tag Label (17161,)
encode num  100  done
encode num  200  done
encode num  300  done
encode num  400  done
encode num  500  done
encode num  600  done
encode num  700  done
encode num  800  done
encode num  900  done
encode num  1000  done
encode num  1100  done
encode num  1200  done
encode num  1300  done
encode num  1400  done
encode num  1500  done
encode num  1600  done
encode num  1700  done
encode num  1800  done
encode num  1900  done
encode num  2000  done
encode num  2100  done
encode num  2200  done
encode num  2300  done
encode num  2400  done
encode num  2500  done
onehot fit all tags
all tag one hot (17161, 51)
encode num  100  done
results: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0

results: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

encode num  1800  done
results: [[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

## normalize

In [710]:
scale_duration=pd.Series(preprocessing.scale(np.array(EDA_data.duration)))
scale_languages=pd.Series(preprocessing.scale(np.array(EDA_data.languages)))

c:\program files\python35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


## get xs together

In [784]:
EDA_done=pd.concat([nonlinear_title_sentiment,nonlinear_description_sentiment,scale_duration,one_hot_event,one_hot_sum_tags], axis=1)
EDA_done.shape

(2550, 65)

## preprocess ys

In [658]:
EDA_data['q']=0
EDA_data.loc[EDA_data.views>EDA_data.views.quantile(0.25),'q']=1
EDA_data.loc[EDA_data.views>EDA_data.views.quantile(0.5),'q']=2
EDA_data.loc[EDA_data.views>EDA_data.views.quantile(0.75),'q']=3
EDA_data.head()

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\program files\python35\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,title,description,event,duration,languages,tags,ratings,views,clean_tags,q
0,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,others,1164,60,"['children', 'creativity', 'culture', 'dance',...","[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",47227110,"[children, creativity, culture, dance, educati...",3
1,Averting the climate crisis,With the same humor and humanity he exuded in ...,others,977,43,"['alternative energy', 'cars', 'climate change...","[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",3200520,"[alternativeenergy, cars, climatechange, cultu...",3
2,Simplicity sells,New York Times columnist David Pogue takes aim...,others,1286,26,"['computers', 'entertainment', 'interface desi...","[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",1636292,"[computers, entertainment, interfacedesign, me...",2
3,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",others,1116,35,"['MacArthur grant', 'activism', 'business', 'c...","[{'id': 3, 'name': 'Courageous', 'count': 760}...",1697550,"[MacArthurgrant, activism, business, cities, e...",2
4,The best stats you've ever seen,You've never seen data presented like this. Wi...,others,1190,48,"['Africa', 'Asia', 'Google', 'demo', 'economic...","[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",12005869,"[Africa, Asia, Google, demo, economics, global...",3


In [704]:
scale_views=pd.Series(preprocessing.scale(np.array(EDA_data['views'])))

c:\program files\python35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


## combine xs、ys

In [785]:
EDA_done_withY=pd.concat([EDA_done,one_hot_q],axis=1)


In [742]:
def get_train_test(csv):
    msk = np.random.rand(len(csv))<0.8
    train = csv[msk]
    test = csv[~msk]
    return(train,test)
def get_x_y(train,xs_count,ys_count):
    x_train = np.array(train.iloc[0:, 0:xs_count])
    y_train = np.array(train.iloc[0:,xs_count:xs_count+ys_count]).reshape(len(train),ys_count)
    return(x_train,y_train)

In [786]:
XS_COUNT=65
YS_COUNT=4
train,test=get_train_test(EDA_done_withY)
# train.loc[:,]
x_train,y_train=get_x_y(train,XS_COUNT,YS_COUNT)
x_test,y_test=get_x_y(test,XS_COUNT,YS_COUNT)
y_train

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       ..., 
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=uint8)

In [796]:
model = Sequential()
# model.add(Dense(units=100,
#                input_dim=13,
#                kernel_initializer='uniform',
#                activation='relu'))
model.add(Dense(units=50,
                input_dim=XS_COUNT,
               kernel_initializer='uniform',
               activation='tanh'))
model.add(Dropout(rate=0.15))
model.add(Dense(units=YS_COUNT,
               kernel_initializer='uniform',
               activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])



NameError: name 'regularizers' is not defined

In [794]:
train_history = model.fit(x=x_train, 
                          y=y_train,
                          validation_split=0.05,
                          epochs=100, 
                          batch_size=30,
                          verbose=1)

Train on 1919 samples, validate on 101 samples
Epoch 1/100
1919/1919 [==============================] - 2s 1ms/step - loss: 1.3815 - acc: 0.2960 - val_loss: 1.4267 - val_acc: 0.3366
Epoch 2/100
1919/1919 [==============================] - 0s 107us/step - loss: 1.3664 - acc: 0.3210 - val_loss: 1.4404 - val_acc: 0.2772
Epoch 3/100
1919/1919 [==============================] - 0s 122us/step - loss: 1.3577 - acc: 0.3340 - val_loss: 1.4737 - val_acc: 0.2574
Epoch 4/100
1919/1919 [==============================] - 0s 127us/step - loss: 1.3408 - acc: 0.3580 - val_loss: 1.4358 - val_acc: 0.2772
Epoch 5/100
1919/1919 [==============================] - 0s 126us/step - loss: 1.3264 - acc: 0.3710 - val_loss: 1.4031 - val_acc: 0.2574
Epoch 6/100
1919/1919 [==============================] - 0s 117us/step - loss: 1.3177 - acc: 0.3705 - val_loss: 1.3445 - val_acc: 0.2970
Epoch 7/100
1919/1919 [==============================] - 0s 129us/step - loss: 1.3092 - acc: 0.3721 - val_loss: 1.4626 - val_acc: 0.2

1919/1919 [==============================] - 0s 127us/step - loss: 1.2462 - acc: 0.4289 - val_loss: 1.4816 - val_acc: 0.2475
Epoch 61/100
1919/1919 [==============================] - 0s 123us/step - loss: 1.2510 - acc: 0.4153 - val_loss: 1.4089 - val_acc: 0.2574
Epoch 62/100
1919/1919 [==============================] - 0s 130us/step - loss: 1.2403 - acc: 0.4164 - val_loss: 1.3926 - val_acc: 0.2376
Epoch 63/100
1919/1919 [==============================] - 0s 132us/step - loss: 1.2469 - acc: 0.4221 - val_loss: 1.5157 - val_acc: 0.2475
Epoch 64/100
1919/1919 [==============================] - 0s 135us/step - loss: 1.2396 - acc: 0.4257 - val_loss: 1.4171 - val_acc: 0.2079
Epoch 65/100
1919/1919 [==============================] - 0s 141us/step - loss: 1.2503 - acc: 0.4174 - val_loss: 1.4101 - val_acc: 0.2574
Epoch 66/100
1919/1919 [==============================] - 0s 137us/step - loss: 1.2477 - acc: 0.4242 - val_loss: 1.4568 - val_acc: 0.2574
Epoch 67/100
1919/1919 [=======================

In [795]:
scores = model.evaluate(x_test,y_test)
scores

530/530 [==============================] - 0s 54us/step


[1.3488799990348097, 0.34339622647132512]

In [797]:
!jupyter nbconvert --to script Untitled.ipynb

[NbConvertApp] Converting notebook Untitled.ipynb to script
[NbConvertApp] Writing 7654 bytes to Untitled.py
